## 112 신고

In [3]:
import geopandas as gpd
import pandas as pd
import folium
from folium import plugins
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

/opt/app-root/lib/python3.6/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


+ 2018년 6월 2일부터만 존재하는 데이터

In [4]:
call = pd.read_csv('5.112신고.csv')
call

,date,jur_stn,report_sx,inc_info
0,20180603.0,서울서부,남성,분실습득
1,20180603.0,서울송파,여성,폭력
2,20180603.0,서울강서,남성,소음
3,20180603.0,서울영등포,여성,실종(실종아동 등)
4,20180603.0,서울강서,남성,상담문의
...,...,...,...,...
9229106,20210531.0,마산동부,남성,교통사고
9229107,20210531.0,마산중부,남성,가정폭력
9229108,20210531.0,마산중부,불상,화재
9229109,20210531.0,마산동부,남성,기타형사범


In [5]:
call_seoul = call[call['jur_stn'].str.contains('서울', na=False)]
call_sejong = call[call['jur_stn'].str.contains('세종', na=False)]
call_suwon = call[call['jur_stn'].str.contains('수원', na=False)]
call_masan = call[call['jur_stn'].str.contains('마산', na=False)]
call_changwon = call[call['jur_stn'].str.contains('창원', na=False)]
call_jinhae = call[call['jur_stn'].str.contains('진해', na=False)]
call_jinju = call[call['jur_stn'].str.contains('진주', na=False)]

### 결측치 처리

In [6]:
## NA 로 표기된 결측치 조회

print(call_seoul.isnull().sum() / len(call_seoul) * 100)
print(call_sejong.isnull().sum() / len(call_sejong) * 100)
print(call_suwon.isnull().sum() / len(call_suwon) * 100)
print(call_masan.isnull().sum() / len(call_masan) * 100)
print(call_changwon.isnull().sum() / len(call_changwon) * 100)
print(call_jinhae.isnull().sum() / len(call_jinhae) * 100)
print(call_jinju.isnull().sum() / len(call_jinju) * 100)

date         0.000000
jur_stn      0.000000
report_sx    0.023344
inc_info     1.551758
dtype: float64
date         0.000000
jur_stn      0.000000
report_sx    0.035944
inc_info     0.000000
dtype: float64
date         0.000000
jur_stn      0.000000
report_sx    0.013521
inc_info     1.522034
dtype: float64
date         0.000000
jur_stn      0.000000
report_sx    0.037111
inc_info     1.593781
dtype: float64
date         0.000000
jur_stn      0.000000
report_sx    0.038058
inc_info     2.185716
dtype: float64
date         0.000000
jur_stn      0.000000
report_sx    0.043357
inc_info     1.587458
dtype: float64
date         0.000000
jur_stn      0.000000
report_sx    0.031571
inc_info     1.941125
dtype: float64


In [7]:
## NA 결측치는 모든 시에서 2% 미만이므로 삭제함

call_seoul = call_seoul.dropna()
call_sejong = call_sejong.dropna()
call_suwon = call_suwon.dropna()
call_masan = call_masan.dropna()
call_changwon = call_changwon.dropna()
call_jinhae = call_jinhae.dropna()
call_jinju = call_jinju.dropna()

In [8]:
## 내용확인불가에 대한 조회

print(call_seoul.groupby(['inc_info']).count().loc['내용확인불가'] / len(call_seoul))
print(call_sejong.groupby(['inc_info']).count().loc['내용확인불가'] / len(call_sejong))
print(call_suwon.groupby(['inc_info']).count().loc['내용확인불가'] / len(call_suwon))
print(call_masan.groupby(['inc_info']).count().loc['내용확인불가'] / len(call_masan))
print(call_changwon.groupby(['inc_info']).count().loc['내용확인불가'] / len(call_changwon))
print(call_jinhae.groupby(['inc_info']).count().loc['내용확인불가'] / len(call_jinhae))
print(call_jinju.groupby(['inc_info']).count().loc['내용확인불가'] / len(call_jinju))

date         0.0532
jur_stn      0.0532
report_sx    0.0532
Name: 내용확인불가, dtype: float64
date         0.112456
jur_stn      0.112456
report_sx    0.112456
Name: 내용확인불가, dtype: float64
date         0.062543
jur_stn      0.062543
report_sx    0.062543
Name: 내용확인불가, dtype: float64
date         0.096973
jur_stn      0.096973
report_sx    0.096973
Name: 내용확인불가, dtype: float64
date         0.092035
jur_stn      0.092035
report_sx    0.092035
Name: 내용확인불가, dtype: float64
date         0.082091
jur_stn      0.082091
report_sx    0.082091
Name: 내용확인불가, dtype: float64
date         0.121387
jur_stn      0.121387
report_sx    0.121387
Name: 내용확인불가, dtype: float64


In [9]:
## 핫덱 대체를 위한 대체군 조회

print(call_seoul[call_seoul['inc_info']=='내용확인불가']['report_sx'].value_counts())
print(call_sejong[call_sejong['inc_info']=='내용확인불가']['report_sx'].value_counts())
print(call_suwon[call_suwon['inc_info']=='내용확인불가']['report_sx'].value_counts())
print(call_masan[call_masan['inc_info']=='내용확인불가']['report_sx'].value_counts())
print(call_changwon[call_changwon['inc_info']=='내용확인불가']['report_sx'].value_counts())
print(call_jinhae[call_jinhae['inc_info']=='내용확인불가']['report_sx'].value_counts())
print(call_jinju[call_jinju['inc_info']=='내용확인불가']['report_sx'].value_counts())

불상    189502
여성    111725
남성     91752
Name: report_sx, dtype: int64
불상    2826
남성     549
여성     378
Name: report_sx, dtype: int64
불상    22094
여성    14961
남성    14868
Name: report_sx, dtype: int64
불상    9920
남성    8880
여성    5105
Name: report_sx, dtype: int64
불상    10353
남성     7710
여성     7710
Name: report_sx, dtype: int64
불상    3677
남성    2584
여성    1934
Name: report_sx, dtype: int64
불상    8535
여성    8325
남성    8016
Name: report_sx, dtype: int64


In [10]:
## 결측치 처리에 참고가 될만한 변수는 성별밖에 없으므로, 내용확인불가 중 성별까지 불상인 데이터는 삭제한다.

seoul = call_seoul[(call_seoul.report_sx == '불상') & (call_seoul.inc_info == '내용확인불가')].index
call_seoul = call_seoul.drop(seoul)

sejong = call_sejong[(call_sejong.report_sx == '불상') & (call_sejong.inc_info == '내용확인불가')].index
call_sejong = call_sejong.drop(sejong)

suwon = call_suwon[(call_suwon.report_sx == '불상') & (call_suwon.inc_info == '내용확인불가')].index
call_suwon = call_suwon.drop(suwon)

masan = call_masan[(call_masan.report_sx == '불상') & (call_masan.inc_info == '내용확인불가')].index
call_masan = call_masan.drop(masan)

changwon = call_changwon[(call_changwon.report_sx == '불상') & (call_changwon.inc_info == '내용확인불가')].index
call_changwon = call_changwon.drop(changwon)

jinhae = call_jinhae[(call_jinhae.report_sx == '불상') & (call_jinhae.inc_info == '내용확인불가')].index
call_jinhae = call_jinhae.drop(jinhae)

jinju = call_jinju[(call_jinju.report_sx == '불상') & (call_jinju.inc_info == '내용확인불가')].index
call_jinju = call_jinju.drop(jinju)

### 효율적인 결측치 처리를 위해 연도 별로 분류

In [11]:
## date 데이터 타입 변경

call_seoul['date'] = call_seoul['date'].astype(str)
call_sejong['date'] = call_sejong['date'].astype(str)
call_suwon['date'] = call_suwon['date'].astype(str)
call_masan['date'] = call_masan['date'].astype(str)
call_changwon['date'] = call_changwon['date'].astype(str)
call_jinhae['date'] = call_jinhae['date'].astype(str)
call_jinju['date'] = call_jinju['date'].astype(str)

In [12]:
## 연도 슬라이싱

for i in range(len(call_seoul['date'].values)):
    call_seoul['date'].values[i] = call_seoul['date'].values[i][0:6]

for i in range(len(call_sejong['date'].values)):
    call_sejong['date'].values[i] = call_sejong['date'].values[i][0:6]

for i in range(len(call_suwon['date'].values)):
    call_suwon['date'].values[i] = call_suwon['date'].values[i][0:6]
    
for i in range(len(call_changwon['date'].values)):
    call_changwon['date'].values[i] = call_changwon['date'].values[i][0:6]
    
for i in range(len(call_jinhae['date'].values)):
    call_jinhae['date'].values[i] = call_jinhae['date'].values[i][0:6]
    
for i in range(len(call_masan['date'].values)):
    call_masan['date'].values[i] = call_masan['date'].values[i][0:6]
    
for i in range(len(call_jinju['date'].values)):
    call_jinju['date'].values[i] = call_jinju['date'].values[i][0:6]

In [13]:
call_seoul['date'] = call_seoul['date'].astype(int)
call_sejong['date'] = call_sejong['date'].astype(int)
call_suwon['date'] = call_suwon['date'].astype(int)
call_masan['date'] = call_masan['date'].astype(int)
call_changwon['date'] = call_changwon['date'].astype(int)
call_jinhae['date'] = call_jinhae['date'].astype(int)
call_jinju['date'] = call_jinju['date'].astype(int)

In [14]:
seoul_18_down = call_seoul[(call_seoul.date >= 201807) & (call_seoul.date <= 201900)]
seoul_18_down['date'] = '2018 하반기'

seoul_19_up = call_seoul[(call_seoul.date >= 201900) & (call_seoul.date <= 201907)]
seoul_19_up['date'] = '2019 상반기'
seoul_19_down = call_seoul[(call_seoul.date >= 201907) & (call_seoul.date <= 202000)]
seoul_19_down['date'] = '2019 하반기'

seoul_20_up = call_seoul[(call_seoul.date >= 202000) & (call_seoul.date <= 202007)]
seoul_20_up['date'] = '2020 상반기'
seoul_20_down = call_seoul[(call_seoul.date >= 202007) & (call_seoul.date <= 202100)]
seoul_20_down['date'] = '2020 하반기'

call_seoul = pd.concat([seoul_18_down, seoul_19_up, seoul_19_down, seoul_20_up, seoul_20_down])


sejong_18_down = call_sejong[(call_sejong.date >= 201807) & (call_sejong.date <= 201900)]
sejong_18_down['date'] = '2018 하반기'

sejong_19_up = call_sejong[(call_sejong.date >= 201900) & (call_sejong.date <= 201907)]
sejong_19_up['date'] = '2019 상반기'
sejong_19_down = call_sejong[(call_sejong.date >= 201907) & (call_sejong.date <= 202000)]
sejong_19_down['date'] = '2019 하반기'

sejong_20_up = call_sejong[(call_sejong.date >= 202000) & (call_sejong.date <= 202007)]
sejong_20_up['date'] = '2020 상반기'
sejong_20_down = call_sejong[(call_sejong.date >= 202007) & (call_sejong.date <= 202100)]
sejong_20_down['date'] = '2020 하반기'

call_sejong = pd.concat([sejong_18_down, sejong_19_up, sejong_19_down, sejong_20_up, sejong_20_down])


suwon_18_down = call_suwon[(call_suwon.date >= 201807) & (call_suwon.date <= 201900)]
suwon_18_down['date'] = '2018 하반기'

suwon_19_up = call_suwon[(call_suwon.date >= 201900) & (call_suwon.date <= 201907)]
suwon_19_up['date'] = '2019 상반기'
suwon_19_down = call_suwon[(call_suwon.date >= 201907) & (call_suwon.date <= 202000)]
suwon_19_down['date'] = '2019 하반기'

suwon_20_up = call_suwon[(call_suwon.date >= 202000) & (call_suwon.date <= 202007)]
suwon_20_up['date'] = '2020 상반기'
suwon_20_down = call_suwon[(call_suwon.date >= 202007) & (call_suwon.date <= 202100)]
suwon_20_down['date'] = '2020 하반기'

call_suwon = pd.concat([suwon_18_down, suwon_19_up, suwon_19_down, suwon_20_up, suwon_20_down])


masan_18_down = call_masan[(call_masan.date >= 201807) & (call_masan.date <= 201900)]
masan_18_down['date'] = '2018 하반기'

masan_19_up = call_masan[(call_masan.date >= 201900) & (call_masan.date <= 201907)]
masan_19_up['date'] = '2019 상반기'
masan_19_down = call_masan[(call_masan.date >= 201907) & (call_masan.date <= 202000)]
masan_19_down['date'] = '2019 하반기'

masan_20_up = call_masan[(call_masan.date >= 202000) & (call_masan.date <= 202007)]
masan_20_up['date'] = '2020 상반기'
masan_20_down = call_masan[(call_masan.date >= 202007) & (call_masan.date <= 202100)]
masan_20_down['date'] = '2020 하반기'

call_masan = pd.concat([masan_18_down, masan_19_up, masan_19_down, masan_20_up, masan_20_down])


changwon_18_down = call_changwon[(call_changwon.date >= 201807) & (call_changwon.date <= 201900)]
changwon_18_down['date'] = '2018 하반기'

changwon_19_up = call_changwon[(call_changwon.date >= 201900) & (call_changwon.date <= 201907)]
changwon_19_up['date'] = '2019 상반기'
changwon_19_down = call_changwon[(call_changwon.date >= 201907) & (call_changwon.date <= 202000)]
changwon_19_down['date'] = '2019 하반기'

changwon_20_up = call_changwon[(call_changwon.date >= 202000) & (call_changwon.date <= 202007)]
changwon_20_up['date'] = '2020 상반기'
changwon_20_down = call_changwon[(call_changwon.date >= 202007) & (call_changwon.date <= 202100)]
changwon_20_down['date'] = '2020 하반기'

call_changwon = pd.concat([changwon_18_down, changwon_19_up, changwon_19_down, changwon_20_up, changwon_20_down])


jinhae_18_down = call_jinhae[(call_jinhae.date >= 201807) & (call_jinhae.date <= 201900)]
jinhae_18_down['date'] = '2018 하반기'

jinhae_19_up = call_jinhae[(call_jinhae.date >= 201900) & (call_jinhae.date <= 201907)]
jinhae_19_up['date'] = '2019 상반기'
jinhae_19_down = call_jinhae[(call_jinhae.date >= 201907) & (call_jinhae.date <= 202000)]
jinhae_19_down['date'] = '2019 하반기'

jinhae_20_up = call_jinhae[(call_jinhae.date >= 202000) & (call_jinhae.date <= 202007)]
jinhae_20_up['date'] = '2020 상반기'
jinhae_20_down = call_jinhae[(call_jinhae.date >= 202007) & (call_jinhae.date <= 202100)]
jinhae_20_down['date'] = '2020 하반기'

call_jinhae = pd.concat([jinhae_18_down, jinhae_19_up, jinhae_19_down, jinhae_20_up, jinhae_20_down])


jinju_18_down = call_jinju[(call_jinju.date >= 201807) & (call_jinju.date <= 201900)]
jinju_18_down['date'] = '2018 하반기'

jinju_19_up = call_jinju[(call_jinju.date >= 201900) & (call_jinju.date <= 201907)]
jinju_19_up['date'] = '2019 상반기'
jinju_19_down = call_jinju[(call_jinju.date >= 201907) & (call_jinju.date <= 202000)]
jinju_19_down['date'] = '2019 하반기'

jinju_20_up = call_jinju[(call_jinju.date >= 202000) & (call_jinju.date <= 202007)]
jinju_20_up['date'] = '2020 상반기'
jinju_20_down = call_jinju[(call_jinju.date >= 202007) & (call_jinju.date <= 202100)]
jinju_20_down['date'] = '2020 하반기'

call_jinju = pd.concat([jinju_18_down, jinju_19_up, jinju_19_down, jinju_20_up, jinju_20_down])

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

In [15]:
call_seoul_songpa = call_seoul[call_seoul['jur_stn'].str.contains('서울송파', na=False)]
call_seoul_gangseo = call_seoul[call_seoul['jur_stn'].str.contains('서울강서', na=False)]
call_seoul_yungdeongpo = call_seoul[call_seoul['jur_stn'].str.contains('서울영등포', na=False)]
call_seoul_gwanak = call_seoul[call_seoul['jur_stn'].str.contains('서울관악', na=False)]
call_seoul_mapo = call_seoul[call_seoul['jur_stn'].str.contains('서울마포', na=False)]
call_seoul_gangnam = call_seoul[call_seoul['jur_stn'].str.contains('서울강남', na=False)]
call_seoul_jungrang = call_seoul[call_seoul['jur_stn'].str.contains('서울중랑', na=False)]
call_seoul_nowon = call_seoul[call_seoul['jur_stn'].str.contains('서울노원', na=False)]
call_seoul_guro = call_seoul[call_seoul['jur_stn'].str.contains('서울구로', na=False)]
call_seoul_gangdong = call_seoul[call_seoul['jur_stn'].str.contains('서울강동', na=False)]
call_seoul_gwangjin = call_seoul[call_seoul['jur_stn'].str.contains('서울광진', na=False)]
call_seoul_dongdaemun = call_seoul[call_seoul['jur_stn'].str.contains('서울동대문', na=False)]
call_seoul_seocho = call_seoul[call_seoul['jur_stn'].str.contains('서울서초', na=False)]
call_seoul_gangbuk = call_seoul[call_seoul['jur_stn'].str.contains('서울강북', na=False)]
call_seoul_yangchun = call_seoul[call_seoul['jur_stn'].str.contains('서울양천', na=False)]
call_seoul_yongsan = call_seoul[call_seoul['jur_stn'].str.contains('서울용산', na=False)]
call_seoul_suseo = call_seoul[call_seoul['jur_stn'].str.contains('서울수서', na=False)]
call_seoul_dongjak = call_seoul[call_seoul['jur_stn'].str.contains('서울동작', na=False)]
call_seoul_seodaemun = call_seoul[call_seoul['jur_stn'].str.contains('서울서대문', na=False)]
call_seoul_geumchun = call_seoul[call_seoul['jur_stn'].str.contains('서울금천', na=False)]
call_seoul_sungdong = call_seoul[call_seoul['jur_stn'].str.contains('서울성동', na=False)]
call_seoul_eunpyeong = call_seoul[call_seoul['jur_stn'].str.contains('서울은평', na=False)]
call_seoul_dobong = call_seoul[call_seoul['jur_stn'].str.contains('서울도봉', na=False)]
call_seoul_jungbu = call_seoul[call_seoul['jur_stn'].str.contains('서울중부', na=False)]
call_seoul_sungbuk = call_seoul[call_seoul['jur_stn'].str.contains('서울성북', na=False)]
call_seoul_jongro = call_seoul[call_seoul['jur_stn'].str.contains('서울종로', na=False)]
call_seoul_seobu = call_seoul[call_seoul['jur_stn'].str.contains('서울서부', na=False)]
call_seoul_hyehwa = call_seoul[call_seoul['jur_stn'].str.contains('서울혜화', na=False)]
call_seoul_jongam = call_seoul[call_seoul['jur_stn'].str.contains('서울종암', na=False)]
call_seoul_namdaemun = call_seoul[call_seoul['jur_stn'].str.contains('서울남대문', na=False)]
call_seoul_bangbae = call_seoul[call_seoul['jur_stn'].str.contains('서울방배', na=False)]

In [16]:
def na_112(df):
    
    ## df의 18년도 하반기 결측치
    down_18_na = df[(df.date=='2018 하반기') & (df.inc_info=='내용확인불가')]
    down_18_na_male = down_18_na[down_18_na.report_sx=='남성']
    down_18_na_female = down_18_na[down_18_na.report_sx=='여성']
    
    ## df의 18년도 하반기 결측치를 제거한 모집단이 될 데이터
    down_18 = df[df.date=='2018 하반기'].drop(down_18_na.index)
    
    ## 성별 분류
    down_18_male = down_18[down_18.report_sx=='남성']
    down_18_female = down_18[down_18.report_sx=='여성']
    down_18_unknown = down_18[down_18.report_sx=='불상']
    
    ## 사건 별 확률 정의
    down_18_male_prob = list((down_18_male.value_counts() / sum(down_18_male.value_counts().values)).values)
    down_18_female_prob = list((down_18_female.value_counts() / sum(down_18_female.value_counts().values)).values)
    
    
    while 1:
    ## 다항분포를 이용한 결측치 구하기
        multi_down_18_male = np.random.multinomial(len(down_18_na_male), pvals=down_18_male_prob, size=30)

        down_18_na_replace_male = []
        
        
        ## 대체값 리스트 생성
        for i in range(len(down_18_male.value_counts('inc_info').index)):
            a = np.mean(multi_down_18_male[:,i])
            down_18_na_replace_male.append(a)

            
        ## 반올림을 하면서 값이 변하기 때문에 실제 결측치 개수와, 대체값의 개수를 맞춰줘야함
        if sum(np.round(down_18_na_replace_male)) == len(down_18_na_male):
            down_18_na_replace_male = np.round(down_18_na_replace_male)
            break

        else:
            continue
        

    while 1:
        ## 다항분포를 이용한 결측치 구하기
        multi_down_18_female = np.random.multinomial(len(down_18_na_female), pvals=down_18_female_prob, size=30)

        down_18_na_replace_female = []

        ## 대체값 리스트 생성
        for i in range(len(down_18_female.value_counts('inc_info').index)):
            a = np.mean(multi_down_18_female[:,i])
            down_18_na_replace_female.append(a)
    
        ## 반올림을 하면서 값이 변하기 때문에 실제 결측치 개수와, 대체값의 개수를 맞춰줘야함
        if sum(np.round(down_18_na_replace_female)) == len(down_18_na_female):
            down_18_na_replace_female = np.round(down_18_na_replace_female)
            break

        else:
            continue
            
    ### down_18_na_replace_male, down_18_na_replace_female 을 결측치 자리에 넣어줘야함.
    ## 이중반복문을 이용해 결측치 대체값을 데이터프레임에 넣어준다.
        
    complete_list_male = []
    for i in range(len(down_18_na_replace_male)):
        for j in range(int(down_18_na_replace_male[i])):
            complete_list_male.append(down_18_male.value_counts().keys()[i][3])
                
    down_18_na_male['inc_info'] = complete_list_male
    complete_down_18_male = pd.concat([down_18_na_male, down_18_male])
    
    complete_list_female = []
    for i in range(len(down_18_na_replace_female)):
        for j in range(int(down_18_na_replace_female[i])):
            complete_list_female.append(down_18_female.value_counts().keys()[i][3])
                
    down_18_na_female['inc_info'] = complete_list_female
    complete_down_18_female = pd.concat([down_18_na_female, down_18_female])
    
    complete_down_18 = pd.concat([complete_down_18_male, complete_down_18_female, down_18_unknown])

    ################################19 상반기############################################
    ## df의 19년도 상반기 결측치
    up_19_na = df[(df.date=='2019 상반기') & (df.inc_info=='내용확인불가')]
    up_19_na_male = up_19_na[up_19_na.report_sx=='남성']
    up_19_na_female = up_19_na[up_19_na.report_sx=='여성']
    
    ## df의 19년도 상반기 결측치를 제거한 모집단이 될 데이터
    up_19 = df[df.date=='2019 상반기'].drop(up_19_na.index)
    
    ## 성별 분류
    up_19_male = up_19[up_19.report_sx=='남성']
    up_19_female = up_19[up_19.report_sx=='여성']
    up_19_unknown = up_19[up_19.report_sx=='불상']
    
    ## 사건 별 확률 정의
    up_19_male_prob = list((up_19_male.value_counts() / sum(up_19_male.value_counts().values)).values)
    up_19_female_prob = list((up_19_female.value_counts() / sum(up_19_female.value_counts().values)).values)
    
    
    while 1:
    ## 다항분포를 이용한 결측치 구하기
        multi_up_19_male = np.random.multinomial(len(up_19_na_male), pvals=up_19_male_prob, size=30)

        up_19_na_replace_male = []
        
        
        ## 대체값 리스트 생성
        for i in range(len(up_19_male.value_counts('inc_info').index)):
            a = np.mean(multi_up_19_male[:,i])
            up_19_na_replace_male.append(a)

            
        ## 반올림을 하면서 값이 변하기 때문에 실제 결측치 개수와, 대체값의 개수를 맞춰줘야함
        if sum(np.round(up_19_na_replace_male)) == len(up_19_na_male):
            up_19_na_replace_male = np.round(up_19_na_replace_male)
            break

        else:
            continue
        

    while 1:
        ## 다항분포를 이용한 결측치 구하기
        multi_up_19_female = np.random.multinomial(len(up_19_na_female), pvals=up_19_female_prob, size=30)

        up_19_na_replace_female = []

        ## 대체값 리스트 생성
        for i in range(len(up_19_female.value_counts('inc_info').index)):
            a = np.mean(multi_up_19_female[:,i])
            up_19_na_replace_female.append(a)
    
        ## 반올림을 하면서 값이 변하기 때문에 실제 결측치 개수와, 대체값의 개수를 맞춰줘야함
        if sum(np.round(up_19_na_replace_female)) == len(up_19_na_female):
            up_19_na_replace_female = np.round(up_19_na_replace_female)
            break

        else:
            continue
            
    ### up_19_na_replace_male, up_19_na_replace_female 을 결측치 자리에 넣어줘야함.
    ## 이중반복문을 이용해 결측치 대체값을 데이터프레임에 넣어준다.
        
    complete_list_male = []
    for i in range(len(up_19_na_replace_male)):
        for j in range(int(up_19_na_replace_male[i])):
            complete_list_male.append(up_19_male.value_counts().keys()[i][3])
                
    up_19_na_male['inc_info'] = complete_list_male
    complete_up_19_male = pd.concat([up_19_na_male, up_19_male])
    
    complete_list_female = []
    for i in range(len(up_19_na_replace_female)):
        for j in range(int(up_19_na_replace_female[i])):
            complete_list_female.append(up_19_female.value_counts().keys()[i][3])
                
    up_19_na_female['inc_info'] = complete_list_female
    complete_up_19_female = pd.concat([up_19_na_female, up_19_female])
    
    complete_up_19 = pd.concat([complete_up_19_male, complete_up_19_female, up_19_unknown])

    ##########################19 하반기##############################
    ## df의 19년도 하반기 결측치
    down_19_na = df[(df.date=='2019 하반기') & (df.inc_info=='내용확인불가')]
    down_19_na_male = down_19_na[down_19_na.report_sx=='남성']
    down_19_na_female = down_19_na[down_19_na.report_sx=='여성']
    
    ## df의 19년도 하반기 결측치를 제거한 모집단이 될 데이터
    down_19 = df[df.date=='2019 하반기'].drop(down_19_na.index)
    
    ## 성별 분류
    down_19_male = down_19[down_19.report_sx=='남성']
    down_19_female = down_19[down_19.report_sx=='여성']
    down_19_unknown = down_19[down_19.report_sx=='불상']
    
    ## 사건 별 확률 정의
    down_19_male_prob = list((down_19_male.value_counts() / sum(down_19_male.value_counts().values)).values)
    down_19_female_prob = list((down_19_female.value_counts() / sum(down_19_female.value_counts().values)).values)
    
    
    while 1:
    ## 다항분포를 이용한 결측치 구하기
        multi_down_19_male = np.random.multinomial(len(down_19_na_male), pvals=down_19_male_prob, size=30)

        down_19_na_replace_male = []
        
        
        ## 대체값 리스트 생성
        for i in range(len(down_19_male.value_counts('inc_info').index)):
            a = np.mean(multi_down_19_male[:,i])
            down_19_na_replace_male.append(a)

            
        ## 반올림을 하면서 값이 변하기 때문에 실제 결측치 개수와, 대체값의 개수를 맞춰줘야함
        if sum(np.round(down_19_na_replace_male)) == len(down_19_na_male):
            down_19_na_replace_male = np.round(down_19_na_replace_male)
            break

        else:
            continue
        

    while 1:
        ## 다항분포를 이용한 결측치 구하기
        multi_down_19_female = np.random.multinomial(len(down_19_na_female), pvals=down_19_female_prob, size=30)

        down_19_na_replace_female = []

        ## 대체값 리스트 생성
        for i in range(len(down_19_female.value_counts('inc_info').index)):
            a = np.mean(multi_down_19_female[:,i])
            down_19_na_replace_female.append(a)
    
        ## 반올림을 하면서 값이 변하기 때문에 실제 결측치 개수와, 대체값의 개수를 맞춰줘야함
        if sum(np.round(down_19_na_replace_female)) == len(down_19_na_female):
            down_19_na_replace_female = np.round(down_19_na_replace_female)
            break

        else:
            continue
            
    ### down_19_na_replace_male, down_19_na_replace_female 을 결측치 자리에 넣어줘야함.
    ## 이중반복문을 이용해 결측치 대체값을 데이터프레임에 넣어준다.
        
    complete_list_male = []
    for i in range(len(down_19_na_replace_male)):
        for j in range(int(down_19_na_replace_male[i])):
            complete_list_male.append(down_19_male.value_counts().keys()[i][3])
                
    down_19_na_male['inc_info'] = complete_list_male
    complete_down_19_male = pd.concat([down_19_na_male, down_19_male])
    
    complete_list_female = []
    for i in range(len(down_19_na_replace_female)):
        for j in range(int(down_19_na_replace_female[i])):
            complete_list_female.append(down_19_female.value_counts().keys()[i][3])
                
    down_19_na_female['inc_info'] = complete_list_female
    complete_down_19_female = pd.concat([down_19_na_female, down_19_female])
    
    complete_down_19 = pd.concat([complete_down_19_male, complete_down_19_female, down_19_unknown])

    ##########################20 상반기##############################
    ## df의 20년도 상반기 결측치
    up_20_na = df[(df.date=='2020 상반기') & (df.inc_info=='내용확인불가')]
    up_20_na_male = up_20_na[up_20_na.report_sx=='남성']
    up_20_na_female = up_20_na[up_20_na.report_sx=='여성']
    
    ## df의 20년도 상반기 결측치를 제거한 모집단이 될 데이터
    up_20 = df[df.date=='2020 상반기'].drop(up_20_na.index)
    
    ## 성별 분류
    up_20_male = up_20[up_20.report_sx=='남성']
    up_20_female = up_20[up_20.report_sx=='여성']
    up_20_unknown = up_20[up_20.report_sx=='불상']
    
    ## 사건 별 확률 정의
    up_20_male_prob = list((up_20_male.value_counts() / sum(up_20_male.value_counts().values)).values)
    up_20_female_prob = list((up_20_female.value_counts() / sum(up_20_female.value_counts().values)).values)
    
    
    while 1:
    ## 다항분포를 이용한 결측치 구하기
        multi_up_20_male = np.random.multinomial(len(up_20_na_male), pvals=up_20_male_prob, size=30)

        up_20_na_replace_male = []
        
        
        ## 대체값 리스트 생성
        for i in range(len(up_20_male.value_counts('inc_info').index)):
            a = np.mean(multi_up_20_male[:,i])
            up_20_na_replace_male.append(a)

            
        ## 반올림을 하면서 값이 변하기 때문에 실제 결측치 개수와, 대체값의 개수를 맞춰줘야함
        if sum(np.round(up_20_na_replace_male)) == len(up_20_na_male):
            up_20_na_replace_male = np.round(up_20_na_replace_male)
            break

        else:
            continue
        

    while 1:
        ## 다항분포를 이용한 결측치 구하기
        multi_up_20_female = np.random.multinomial(len(up_20_na_female), pvals=up_20_female_prob, size=30)

        up_20_na_replace_female = []

        ## 대체값 리스트 생성
        for i in range(len(up_20_female.value_counts('inc_info').index)):
            a = np.mean(multi_up_20_female[:,i])
            up_20_na_replace_female.append(a)
    
        ## 반올림을 하면서 값이 변하기 때문에 실제 결측치 개수와, 대체값의 개수를 맞춰줘야함
        if sum(np.round(up_20_na_replace_female)) == len(up_20_na_female):
            up_20_na_replace_female = np.round(up_20_na_replace_female)
            break

        else:
            continue
            
    ### up_20_na_replace_male, up_20_na_replace_female 을 결측치 자리에 넣어줘야함.
    ## 이중반복문을 이용해 결측치 대체값을 데이터프레임에 넣어준다.
        
    complete_list_male = []
    for i in range(len(up_20_na_replace_male)):
        for j in range(int(up_20_na_replace_male[i])):
            complete_list_male.append(up_20_male.value_counts().keys()[i][3])
                
    up_20_na_male['inc_info'] = complete_list_male
    complete_up_20_male = pd.concat([up_20_na_male, up_20_male])
    
    complete_list_female = []
    for i in range(len(up_20_na_replace_female)):
        for j in range(int(up_20_na_replace_female[i])):
            complete_list_female.append(up_20_female.value_counts().keys()[i][3])
                
    up_20_na_female['inc_info'] = complete_list_female
    complete_up_20_female = pd.concat([up_20_na_female, up_20_female])
    
    complete_up_20 = pd.concat([complete_up_20_male, complete_up_20_female, up_20_unknown])

    #############################20 하반기###########################
    ## df의 20년도 하반기 결측치
    down_20_na = df[(df.date=='2020 하반기') & (df.inc_info=='내용확인불가')]
    down_20_na_male = down_20_na[down_20_na.report_sx=='남성']
    down_20_na_female = down_20_na[down_20_na.report_sx=='여성']
    
    ## df의 20년도 하반기 결측치를 제거한 모집단이 될 데이터
    down_20 = df[df.date=='2020 하반기'].drop(down_20_na.index)
    
    ## 성별 분류
    down_20_male = down_20[down_20.report_sx=='남성']
    down_20_female = down_20[down_20.report_sx=='여성']
    down_20_unknown = down_20[down_20.report_sx=='불상']
    
    ## 사건 별 확률 정의
    down_20_male_prob = list((down_20_male.value_counts() / sum(down_20_male.value_counts().values)).values)
    down_20_female_prob = list((down_20_female.value_counts() / sum(down_20_female.value_counts().values)).values)
    
    
    while 1:
    ## 다항분포를 이용한 결측치 구하기
        multi_down_20_male = np.random.multinomial(len(down_20_na_male), pvals=down_20_male_prob, size=30)

        down_20_na_replace_male = []
        
        
        ## 대체값 리스트 생성
        for i in range(len(down_20_male.value_counts('inc_info').index)):
            a = np.mean(multi_down_20_male[:,i])
            down_20_na_replace_male.append(a)

            
        ## 반올림을 하면서 값이 변하기 때문에 실제 결측치 개수와, 대체값의 개수를 맞춰줘야함
        if sum(np.round(down_20_na_replace_male)) == len(down_20_na_male):
            down_20_na_replace_male = np.round(down_20_na_replace_male)
            break

        else:
            continue
        

    while 1:
        ## 다항분포를 이용한 결측치 구하기
        multi_down_20_female = np.random.multinomial(len(down_20_na_female), pvals=down_20_female_prob, size=30)

        down_20_na_replace_female = []

        ## 대체값 리스트 생성
        for i in range(len(down_20_female.value_counts('inc_info').index)):
            a = np.mean(multi_down_20_female[:,i])
            down_20_na_replace_female.append(a)
    
        ## 반올림을 하면서 값이 변하기 때문에 실제 결측치 개수와, 대체값의 개수를 맞춰줘야함
        if sum(np.round(down_20_na_replace_female)) == len(down_20_na_female):
            down_20_na_replace_female = np.round(down_20_na_replace_female)
            break

        else:
            continue
            
    ### down_20_na_replace_male, down_20_na_replace_female 을 결측치 자리에 넣어줘야함.
    ## 이중반복문을 이용해 결측치 대체값을 데이터프레임에 넣어준다.
        
    complete_list_male = []
    for i in range(len(down_20_na_replace_male)):
        for j in range(int(down_20_na_replace_male[i])):
            complete_list_male.append(down_20_male.value_counts().keys()[i][3])
                
    down_20_na_male['inc_info'] = complete_list_male
    complete_down_20_male = pd.concat([down_20_na_male, down_20_male])
    
    complete_list_female = []
    for i in range(len(down_20_na_replace_female)):
        for j in range(int(down_20_na_replace_female[i])):
            complete_list_female.append(down_20_female.value_counts().keys()[i][3])
                
    down_20_na_female['inc_info'] = complete_list_female
    complete_down_20_female = pd.concat([down_20_na_female, down_20_female])
    
    complete_down_20 = pd.concat([complete_down_20_male, complete_down_20_female, down_20_unknown])
    
    return pd.concat([complete_down_18,complete_up_19,complete_down_19,complete_up_20,complete_down_20])

In [17]:
# 서울시 전체 결측치 처리

call_seoul_songpa = na_112(call_seoul_songpa)
call_seoul_gangseo = na_112(call_seoul_gangseo)
call_seoul_yungdeongpo = na_112(call_seoul_yungdeongpo)
call_seoul_gwanak = na_112(call_seoul_gwanak)
call_seoul_mapo = na_112(call_seoul_mapo)
call_seoul_gangnam = na_112(call_seoul_gangnam)
call_seoul_jungrang = na_112(call_seoul_jungrang)
call_seoul_nowon = na_112(call_seoul_nowon)
call_seoul_guro = na_112(call_seoul_guro)
call_seoul_gangdong = na_112(call_seoul_gangdong)
call_seoul_gwangjin = na_112(call_seoul_gwangjin)
call_seoul_dongdaemun =na_112(call_seoul_dongdaemun)
call_seoul_seocho = na_112(call_seoul_seocho)
call_seoul_gangbuk = na_112(call_seoul_gangbuk)
call_seoul_yangchun = na_112(call_seoul_yangchun)
call_seoul_yongsan = na_112(call_seoul_yongsan)
call_seoul_suseo = na_112(call_seoul_suseo)
call_seoul_dongjak = na_112(call_seoul_dongjak)
call_seoul_seodaemun = na_112(call_seoul_seodaemun)
call_seoul_geumchun = na_112(call_seoul_geumchun)
call_seoul_sungdong = na_112(call_seoul_sungdong)
call_seoul_eunpyeong = na_112(call_seoul_eunpyeong)
call_seoul_dobong = na_112(call_seoul_dobong)
call_seoul_jungbu = na_112(call_seoul_jungbu)
call_seoul_sungbuk = na_112(call_seoul_sungbuk)
call_seoul_jongro = na_112(call_seoul_jongro)
call_seoul_seobu = na_112(call_seoul_seobu)
call_seoul_hyehwa = na_112(call_seoul_hyehwa)
call_seoul_jongam = na_112(call_seoul_jongam)
call_seoul_namdaemun = na_112(call_seoul_namdaemun)
call_seoul_bangbae = na_112(call_seoul_bangbae)

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

In [ ]:
## 세종시 처리

call_sejong = na_112(call_sejong)

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

In [ ]:
call_suwon

In [ ]:
## 수원시 관할서 별 분류

call_suwon_nambu = call_suwon[call_suwon['jur_stn'].str.contains('', na=False)]
call_suwon_jungbu = call_suwon[call_suwon['jur_stn'].str.contains('', na=False)]
call_suwon_seobu = call_suwon[call_suwon['jur_stn'].str.contains('', na=False)]

## 결측치 처리

call_suwon_nambu = na_112(call_suwon_nambu)
call_suwon_jungbu = na_112(call_suwon_jungbu)
call_suwon_seobu = na_112(call_suwon_seobu)

In [ ]:
call_masan

In [ ]:
## 마산 관할서 별 분류

call_masan_dongbu = call_masan[call_masan['jur_stn'].str.contains('', na=False)]
call_masan_jungbu = call_masan[call_masan['jur_stn'].str.contains('', na=False)]

## 결측치 처리

call_masan_dongbu = na_112(call_masan_dongbu)
call_masan_jungbu = na_112(call_masan_jungbu)

In [ ]:
## 창원 관할서 별 분류

call_changwon_seobu = call_changwon[call_changwon['jur_stn'].str.contains('', na=False)]
call_changwon_jungbu = call_changwon[call_changwon['jur_stn'].str.contains('', na=False)]

## 결측치 처리

call_changwon_seobu = na_112(call_changwon_seobu)
call_changwon_jungbu = na_112(call_changwon_jungbu)

In [ ]:
## 진해시 처리

call_jinhae = na_112(call_jinhae)

In [ ]:
## 진주시 처리

call_jinju = na_112(call_jinju)